In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

data_dir = Path("../data")
df = pd.read_csv(data_dir / "stocks_clean.csv", parse_dates=["Date"])
df.head()

In [ ]:
def add_indicators(group, short=20, long=50):
    group = group.sort_values("Date")
    group["SMA_short"] = group["Close"].rolling(window=short).mean()
    group["SMA_long"] = group["Close"].rolling(window=long).mean()
    
    delta = group["Close"].diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    window = 14
    avg_gain = gain.rolling(window=window, min_periods=window).mean()
    avg_loss = loss.rolling(window=window, min_periods=window).mean()
    rs = avg_gain / avg_loss
    group["RSI"] = 100 - (100 / (1 + rs))
    
    exp1 = group["Close"].ewm(span=12, adjust=False).mean()
    exp2 = group["Close"].ewm(span=26, adjust=False).mean()
    group["MACD"] = exp1 - exp2
    group["Signal"] = group["MACD"].ewm(span=9, adjust=False).mean()
    return group

df_ind = df.groupby("Ticker", group_keys=False).apply(add_indicators)
df_ind.to_csv(data_dir / "stocks_with_indicators.csv", index=False)
df_ind.tail()